# Mod 4 Project - Starter Notebook

This notebook has been provided to you so that you can make use of the following starter code to help with the trickier parts of preprocessing the Zillow dataset. 

The notebook contains a rough outline the general order you'll likely want to take in this project. You'll notice that most of the areas are left blank. This is so that it's more obvious exactly when you should make use of the starter code provided for preprocessing. 

**_NOTE:_** The number of empty cells are not meant to infer how much or how little code should be involved in any given step--we've just provided a few for your convenience. Add, delete, and change things around in this notebook as needed!

# Some Notes Before Starting

This project will be one of the more challenging projects you complete in this program. This is because working with Time Series data is a bit different than working with regular datasets. In order to make this a bit less frustrating and help you understand what you need to do (and when you need to do it), we'll quickly review the dataset formats that you'll encounter in this project. 

## Wide Format vs Long Format

If you take a look at the format of the data in `zillow_data.csv`, you'll notice that the actual Time Series values are stored as separate columns. Here's a sample: 

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/df_head.png'>

You'll notice that the first seven columns look like any other dataset you're used to working with. However, column 8 refers to the median housing sales values for April 1996, column 9 for May 1996, and so on. This This is called **_Wide Format_**, and it makes the dataframe intuitive and easy to read. However, there are problems with this format when it comes to actually learning from the data, because the data only makes sense if you know the name of the column that the data can be found it. Since column names are metadata, our algorithms will miss out on what dates each value is for. This means that before we pass this data to our ARIMA model, we'll need to reshape our dataset to **_Long Format_**. Reshaped into long format, the dataframe above would now look like:

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/melted1.png'>

There are now many more rows in this dataset--one for each unique time and zipcode combination in the data! Once our dataset is in this format, we'll be able to train an ARIMA model on it. The method used to convert from Wide to Long is `pd.melt()`, and it is common to refer to our dataset as 'melted' after the transition to denote that it is in long format. 

# Helper Functions Provided

Melting a dataset can be tricky if you've never done it before, so you'll see that we have provided a sample function, `melt_data()`, to help you with this step below. Also provided is:

* `get_datetimes()`, a function to deal with converting the column values for datetimes as a pandas series of datetime objects
* Some good parameters for matplotlib to help make your visualizations more readable. 

Good luck!


# Step 1: Load the Data/Filtering for Chosen Zipcodes

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("zillow_data.csv")

In [3]:
df

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14718,58333,1338,Ashfield,MA,Greenfield Town,Franklin,14719,94600.0,94300.0,94000.0,...,216800,217700,218600,218500,218100,216400,213100,209800,209200,209300
14719,59107,3293,Woodstock,NH,Claremont,Grafton,14720,92700.0,92500.0,92400.0,...,202100,208400,212200,215200,214300,213100,213700,218300,222700,225800
14720,75672,40404,Berea,KY,Richmond,Madison,14721,57100.0,57300.0,57500.0,...,121800,122800,124600,126700,128800,130600,131700,132500,133000,133400
14721,93733,81225,Mount Crested Butte,CO,NaN,Gunnison,14722,191100.0,192400.0,193700.0,...,662800,671200,682400,695600,695500,694700,706400,705300,681500,664400


In [66]:
df.iloc[:,160:172]

,2009-01,2009-02,2009-03,2009-04,2009-05,2009-06,2009-07,2009-08,2009-09,2009-10,2009-11,2009-12
7402,54500.0,54400.0,54200.0,54100.0,53900.0,53500.0,53200.0,53000.0,52800.0,52800.0,52900.0,52800.0
13440,220600.0,218100.0,216200.0,213700.0,210500.0,206300.0,202400.0,198700.0,194600.0,189600.0,184300.0,179600.0
5278,85400.0,83200.0,81100.0,79000.0,76800.0,74500.0,72200.0,70000.0,68000.0,66300.0,64900.0,64100.0
3894,49900.0,49800.0,49800.0,49800.0,49700.0,49900.0,50300.0,50500.0,50100.0,49400.0,48700.0,47800.0
2417,131400.0,129100.0,126900.0,124200.0,121400.0,118900.0,117100.0,115600.0,114200.0,113100.0,112600.0,112400.0
...,...,...,...,...,...,...,...,...,...,...,...,...
14669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14674,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14707,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
df['ROI'] = ((df.iloc[:,160:172].mean(axis=1))/(df.iloc[:,260:272].mean(axis=1))-1)*100

In [59]:
df = df.sort_values(['ROI'], ascending=[False])
df[:10]

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,ROI
7402,64354,15902,Johnstown,PA,Johnstown,Cambria,7403,43200.0,43200.0,43300.0,...,28500,28100,27900,27800,27600,27600,27700,28300,29000,88.741917
13440,79884,49713,Boyne Falls,MI,NaN,Charlevoix,13441,NaN,NaN,NaN,...,107000,111000,112400,113700,115600,117800,119300,119300,118000,82.901360
5278,61128,8611,Trenton,NJ,Trenton,Mercer,5279,51400.0,51200.0,50900.0,...,38400,39100,39700,40100,40900,41400,42100,44000,45900,82.502061
3894,88814,70805,Baton Rouge,LA,Baton Rouge,East Baton Rouge,3895,22700.0,22300.0,21900.0,...,27100,26300,25300,25400,26800,27800,28000,28200,28300,82.338537
2417,61129,8618,Trenton,NJ,Trenton,Mercer,2418,74700.0,74200.0,73700.0,...,63500,63500,63700,65400,68600,71500,74500,78400,81100,75.918217
5690,69501,27801,Rocky Mount,NC,Rocky Mount,Nash,5691,44700.0,44900.0,45100.0,...,36500,36500,36200,35900,35600,35300,35700,37200,38500,75.616438
6516,85070,61603,Peoria,IL,Peoria,Peoria,6517,26300.0,26300.0,26400.0,...,29400,28600,28200,28400,28900,28700,29300,31300,33100,71.336328
6881,73733,36078,Tallassee,AL,Montgomery,Elmore,6882,NaN,NaN,NaN,...,96300,96400,96200,95600,95200,95400,96700,100100,103400,65.748842
6321,71480,31217,Macon,GA,Macon,Bibb,6322,47700.0,47800.0,48000.0,...,30000,30100,30500,31000,31800,32400,33400,35300,37200,62.961025
6609,65504,18466,Coolbaugh,PA,East Stroudsburg,Monroe,6610,59900.0,59900.0,59800.0,...,71200,72000,73300,74300,74300,74200,75400,77000,78000,61.022364


In [63]:
df['std']=df.loc[:,'2009-01':'2018-04'].std(skipna=True, axis=1)
#Calculate historical mean value
df['mean']=df.loc[:,'2009-01':'2018-04'].mean(skipna=True, axis=1)
#Calculate coefficient of variance
df['CV']=df['std']/df['mean']
#Define upper limit of CV according to risk profile.
upper_cv = df.CV.quantile(.6)
best_ROI = df[df['CV']<upper_cv].sort_values('ROI',axis=0,ascending=False)[:10]

In [64]:
best_ROI

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,ROI,std,mean,CV
12287,71270,30669,Union Point,GA,NaN,Greene,12288,35000.0,35200.0,35300.0,...,45300,45400,44900,44600,45600,46800,39.254463,5336.695093,55411.607143,0.096310
7359,61026,8244,Somers Point,NJ,Atlantic City,Atlantic,7360,103000.0,102800.0,102600.0,...,161300,164000,166200,167900,169700,170900,37.543700,20165.974579,187638.392857,0.107473
391,71719,31907,Columbus,GA,Columbus,Muscogee,392,69600.0,69400.0,69000.0,...,67600,68200,68200,69200,71900,74100,37.375231,7861.279594,77221.428571,0.101802
14647,61023,8241,Port Republic,NJ,Atlantic City,Atlantic,14648,117700.0,118000.0,118400.0,...,175900,169100,165900,167800,173500,179300,35.216335,18872.329305,206757.142857,0.091278
9276,61016,8225,Northfield,NJ,Atlantic City,Atlantic,9277,107400.0,107600.0,107700.0,...,175000,174800,174900,175000,175100,175300,32.515220,18205.446931,194016.071429,0.093835
8037,60969,8079,Salem,NJ,Philadelphia,Salem,8038,68000.0,68000.0,67900.0,...,93600,92700,91700,92300,95200,97700,31.079301,9353.460971,107579.464286,0.086945
12951,66902,21539,Lonaconing,MD,Cumberland,Allegany,12952,32400.0,32200.0,32100.0,...,46500,46600,45800,46000,47500,48700,30.861544,4231.152375,53622.321429,0.078907
6559,77821,45804,Lima,OH,Lima,Allen,6560,24800.0,24300.0,23800.0,...,24400,24200,23800,23500,24000,24700,30.533832,2442.849828,27665.178571,0.088301
14387,87657,67561,Nickerson,KS,Hutchinson,Reno,14388,41900.0,42000.0,42100.0,...,66500,64300,62900,62400,64200,66900,29.997504,7710.530279,74679.464286,0.103248
14286,70180,28736,Glenville,NC,Cullowhee,Jackson,14287,108000.0,108600.0,109200.0,...,270700,276700,278500,279700,279700,279200,29.663802,29083.121775,296275.892857,0.098162


In [43]:
df[df['ROI'].isna()]

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,ROI
36,61796,10456,New York,NY,New York,Bronx,37,NaN,NaN,NaN,...,357100,356500,357200,362000,368500,374100,379800,388100,394400,NaN
105,84613,60611,Chicago,IL,Chicago,Cook,106,NaN,NaN,NaN,...,1473900,1469500,1472100,1477800,1486000,1497300,1508600,1508100,1493200,NaN
469,87563,67401,Salina,KS,Salina,Saline,470,NaN,NaN,NaN,...,115700,115600,115600,115600,115500,115500,115500,115400,115400,NaN
713,73596,35758,Madison,AL,Huntsville,Madison,714,NaN,NaN,NaN,...,211700,212500,214100,215300,216100,217700,221000,224700,227500,NaN
842,70817,30032,Candler-Mcafee,GA,Atlanta,Dekalb,843,NaN,NaN,NaN,...,107700,111700,114600,118300,122300,125400,128600,132800,135900,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14669,82632,56120,Butterfield,MN,NaN,Watonwan,14670,NaN,NaN,NaN,...,129000,132400,135800,136000,132600,131100,132900,133900,133600,NaN
14674,94662,84726,Escalante,UT,NaN,Garfield,14675,NaN,NaN,NaN,...,181100,181400,181500,180400,179400,178900,178400,177900,177900,NaN
14706,59046,3215,Waterville Valley,NH,Claremont,Grafton,14707,NaN,NaN,NaN,...,780900,774100,767800,778300,796800,808100,811600,806500,796600,NaN
14707,69681,28039,East Spencer,NC,Charlotte,Rowan,14708,NaN,NaN,NaN,...,26400,25500,25100,25100,25600,25800,26600,27700,28300,NaN


In [ ]:
df[(df['City']=="New York"or)]

In [ ]:
majorcities = ['New York','Los Angeles','Houston''Chicago','Las Vegas','Houston','']

In [ ]:
pd.set_option("display.max_rows", None)

In [ ]:
df2.nlargest(n=100, columns=['value'])

In [5]:
majorcities = (df['City'].value_counts()).nlargest(n=30).index.tolist()
majorcities

['New York',
 'Los Angeles',
 'Houston',
 'San Antonio',
 'Washington',
 'Chicago',
 'Austin',
 'Springfield',
 'Philadelphia',
 'Columbus',
 'Las Vegas',
 'Kansas City',
 'Phoenix',
 'Dallas',
 'Portland',
 'San Diego',
 'Louisville',
 'Omaha',
 'Oklahoma City',
 'Denver',
 'Arlington',
 'Jacksonville',
 'San Jose',
 'Richmond',
 'Fort Worth',
 'Indianapolis',
 'Lexington',
 'Charlotte',
 'Madison',
 'Franklin']

In [11]:
#df2.drop(df.loc[df['City']==(city in majorcities)].index, inplace=True)
for city in majorcities:
    df2 = df2[df2['City']==city]
df2.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,variable,value


In [12]:
df2['Metro']

Series([], Name: Metro, dtype: object)

In [ ]:
df2['City'].value_counts()

In [13]:
df2 = pd.melt(df, id_vars = ['RegionID','RegionName','City','State','Metro','CountyName','SizeRank'])
df2

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,variable,value
0,84654,60657,Chicago,IL,Chicago,Cook,1,1996-04,334200.0
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,1996-04,235700.0
2,91982,77494,Katy,TX,Houston,Harris,3,1996-04,210400.0
3,84616,60614,Chicago,IL,Chicago,Cook,4,1996-04,498100.0
4,93144,79936,El Paso,TX,El Paso,El Paso,5,1996-04,77300.0
...,...,...,...,...,...,...,...,...,...
3901590,58333,1338,Ashfield,MA,Greenfield Town,Franklin,14719,2018-04,209300.0
3901591,59107,3293,Woodstock,NH,Claremont,Grafton,14720,2018-04,225800.0
3901592,75672,40404,Berea,KY,Richmond,Madison,14721,2018-04,133400.0
3901593,93733,81225,Mount Crested Butte,CO,NaN,Gunnison,14722,2018-04,664400.0


In [15]:
df2['City'].value_counts()

New York        30210
Los Angeles     25175
Houston         23320
San Antonio     12720
Washington      11925
                ...  
Cleburne          265
Baldwin City      265
Robins            265
Okemos            265
Loma Linda        265
Name: City, Length: 7554, dtype: int64

In [16]:
pd.set_option("display.max_rows", None)

In [21]:
majorcities = (df2['Metro'].value_counts().nlargest(n=20)).index.tolist()
majorcities

['New York',
 'Los Angeles-Long Beach-Anaheim',
 'Chicago',
 'Philadelphia',
 'Washington',
 'Boston',
 'Dallas-Fort Worth',
 'Minneapolis-St Paul',
 'Houston',
 'Pittsburgh',
 'Miami-Fort Lauderdale',
 'Portland',
 'Detroit',
 'Atlanta',
 'Seattle',
 'St. Louis',
 'San Francisco',
 'Kansas City',
 'Phoenix',
 'Baltimore']

In [24]:
for city in majorcities:
    df2.loc[df['Metro'] == city]

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [25]:
df2.sort['SizeRank']

2047     265
12929    265
646      265
6789     265
4740     265
10883    265
8834     265
12928    265
4743     265
6790     265
645      265
2692     265
12931    265
8833     265
10880    265
6791     265
4742     265
2693     265
644      265
12930    265
10881    265
8832     265
8839     265
10886    265
12933    265
643      265
2690     265
2695     265
10882    265
6784     265
8835     265
10895    265
8846     265
12940    265
2699     265
650      265
6793     265
4744     265
12943    265
8845     265
10892    265
4747     265
6794     265
649      265
2696     265
12942    265
10893    265
8844     265
6795     265
4746     265
2697     265
648      265
647      265
2694     265
4741     265
6788     265
4737     265
10887    265
4564     265
638      265
4732     265
10875    265
8826     265
12920    265
4735     265
6782     265
637      265
2684     265
12923    265
8825     265
10872    265
6783     265
4734     265
2685     265
636      265
12922    265
10873    265

# Step 2: Data Preprocessing

In [ ]:
def get_datetimes(df):
    """
    Takes a dataframe:
    returns only those column names that can be converted into datetime objects 
    as datetime objects.
    NOTE number of returned columns may not match total number of columns in passed dataframe
    """
    
    return pd.to_datetime(df.columns.values[1:], format='%Y-%m')

# Step 3: EDA and Visualization

In [ ]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

# NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

# Step 4: Reshape from Wide to Long Format

In [76]:
def melt_data(df):
    """
    Takes the zillow_data dataset in wide form or a subset of the zillow_dataset.  
    Returns a long-form datetime dataframe 
    with the datetime column names as the index and the values as the 'values' column.
    
    If more than one row is passes in the wide-form dataset, the values column
    will be the mean of the values from the datetime columns in all of the rows.
    """
    
    melted = pd.melt(df, id_vars=['RegionName', 'RegionID', 'SizeRank', 'City', 'State', 'Metro', 'CountyName','ROI','std','mean','CV'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

In [77]:
df2=melt_data(df)

In [78]:
df2

,value
time,
1996-04-01,118299.123063
1996-05-01,118419.044139
1996-06-01,118537.423268
1996-07-01,118653.069278
1996-08-01,118780.254312
...,...
2017-12-01,281095.320247
2018-01-01,282657.060382
2018-02-01,284368.688447


# Step 5: ARIMA Modeling

# Step 6: Interpreting Results